In [18]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [6]:
os.chdir("/Users/david/Documents/Informatics/TCGA_Melanoma_RNASeq")
pd.set_option('display.max_columns', None)

In [7]:
sample_metadata = pd.read_csv("data/sample_metadata.csv", usecols = ["sample_id", "tumor_descriptor", "tissue_rollup", "ajcc_pathologic_stage"])
#sample_metadata = pd.read_csv("data/sample_metadata.csv")
sample_metadata.head()

,sample_id,tumor_descriptor,ajcc_pathologic_stage,tissue_rollup
0,mel001,Metastatic,Stage IIIC,Lymph_Nodes
1,mel002,Metastatic,Stage III,Cutanous
2,mel003,Metastatic,Stage I,Lymph_Nodes
3,mel004,Primary,Stage IIC,Cutanous
4,mel005,Metastatic,Stage IV,Cutanous


In [8]:
#epi_mes_score = pd.read_csv("outputs/epi_mes_signature_scores.csv", usecols=lambda x: x != "Unnamed: 0")
epi_mes_score = pd.read_csv("outputs/epi_mes_signature_scores.csv", usecols = ["Sample_ID", "ssGSEA_patient_epi_score", "ssGSEA_patient_mes_score"])
epi_mes_score.head()

,Sample_ID,ssGSEA_patient_epi_score,ssGSEA_patient_mes_score
0,mel001,0.149018,0.601737
1,mel002,0.201855,0.631269
2,mel003,0.157802,0.753784
3,mel004,0.191706,0.554946
4,mel005,0.067251,0.777031


In [9]:
wgcna_module_values = pd.read_csv("outputs/WGCNA_module_eigenvalues.csv")
wgcna_module_values.rename({"Unnamed: 0" : "Sample_ID"}, axis = "columns", inplace = True)
wgcna_module_values.head()

,Sample_ID,MEblue,MEturquoise,MEyellow,MEgreen,MEmagenta,MEbrown,MEsalmon,MElightcyan,MEblack,MEgreenyellow,MEred,MEmidnightblue,MEtan,MEcyan,MEpink,MEpurple,MEgrey
0,mel354,0.020971,0.005956,-0.086223,-0.048382,-0.057826,0.023024,-0.001378,-0.012002,0.023168,-0.033896,-0.035925,0.016831,0.020362,0.051121,-0.036941,0.035503,-0.045974
1,mel432,0.044355,0.020178,0.057323,-0.032854,-0.004383,-0.024271,0.002157,-0.018926,-0.028900,-0.014615,-0.062450,-0.052693,-0.031941,-0.040906,-0.017927,-0.009794,-0.009022
2,mel243,0.028841,-0.008393,0.045201,0.043760,0.036315,-0.022660,0.002516,0.032038,0.055258,0.097136,0.003865,0.044203,0.013156,-0.049340,-0.075039,-0.015590,0.036642
3,mel321,-0.084911,-0.038733,-0.078051,0.028075,-0.022807,0.056430,0.015542,-0.004794,0.020139,-0.006115,-0.000501,-0.003662,0.027545,0.019350,0.050113,0.057358,-0.048247
4,mel218,-0.007241,0.042464,-0.071064,-0.040436,-0.052362,0.012555,-0.015324,-0.017386,-0.008267,-0.037702,0.044787,0.018398,0.018849,0.031989,0.059126,0.018955,-0.093132


In [10]:
deconvolutions = pd.read_csv("outputs/epic_deconvolutions.csv", skiprows = 1)
#deconvolutions.rename({"cell_type" : "Sample_ID"}, axis = "columns", inplace = True)
deconvolutions.columns = ["Sample_ID", "Bcells", "CAFs", "CD4_Tcells", "CD8_Tcells", "Endothelial", "Macrophages", "NK_cells", "Other_cells"]
deconvolutions.head()

,Sample_ID,Bcells,CAFs,CD4_Tcells,CD8_Tcells,Endothelial,Macrophages,NK_cells,Other_cells
0,mel354,1.025579e-10,0.023539,1.045116e-07,7.382235e-08,0.109605,0.005625,3.814432e-05,0.861194
1,mel432,5.479552e-02,0.022260,2.640453e-01,1.779707e-01,0.113383,0.015683,4.243849e-11,0.351862
2,mel243,6.988778e-02,0.019769,2.511736e-01,9.669438e-02,0.149864,0.011853,5.487604e-10,0.400758
3,mel321,5.082280e-09,0.026713,5.343004e-02,3.707480e-08,0.178638,0.007351,6.497756e-07,0.733868
4,mel218,1.586768e-05,0.023621,1.519758e-01,9.125375e-05,0.122105,0.010320,4.908923e-09,0.691871


In [11]:
#could use reduce function
sample_data = pd.merge(epi_mes_score, wgcna_module_values, on = "Sample_ID")
sample_data = pd.merge(sample_data, deconvolutions, on = "Sample_ID")
sample_data.head()


,Sample_ID,ssGSEA_patient_epi_score,ssGSEA_patient_mes_score,MEblue,MEturquoise,MEyellow,MEgreen,MEmagenta,MEbrown,MEsalmon,MElightcyan,MEblack,MEgreenyellow,MEred,MEmidnightblue,MEtan,MEcyan,MEpink,MEpurple,MEgrey,Bcells,CAFs,CD4_Tcells,CD8_Tcells,Endothelial,Macrophages,NK_cells,Other_cells
0,mel001,0.149018,0.601737,-0.027554,0.008308,0.065262,-0.011695,0.006539,-0.007475,0.023422,0.008436,-0.020527,0.006892,-0.033263,0.050509,0.011567,0.001519,0.012380,-0.022201,-0.006652,1.006166e-01,0.022282,0.318466,0.148652,0.165866,0.016016,1.873212e-09,0.228101
1,mel002,0.201855,0.631269,-0.034268,0.011327,0.025679,0.009168,-0.003292,0.001200,0.002366,-0.018266,-0.032988,-0.015256,0.089934,0.015183,-0.023323,0.017897,0.043639,0.039837,-0.038442,4.633967e-02,0.026364,0.223059,0.121218,0.156134,0.014540,2.535559e-10,0.412345
2,mel003,0.157802,0.753784,0.036205,0.020433,0.107598,0.059451,0.042216,-0.096866,0.005888,0.005968,-0.025072,0.050142,0.001949,-0.051039,-0.094073,-0.071653,-0.048965,-0.044652,0.002794,8.955805e-02,0.026561,0.388977,0.195644,0.198778,0.015364,1.378513e-10,0.085119
3,mel004,0.191706,0.554946,-0.094093,-0.053224,-0.016999,-0.008038,-0.064231,0.086740,0.084928,0.026865,-0.023993,-0.017458,0.030773,-0.022214,-0.000901,0.016030,0.041660,0.079763,-0.083695,8.181303e-09,0.030694,0.264179,0.024643,0.135870,0.014467,1.959118e-09,0.530147
4,mel005,0.067251,0.777031,0.040886,0.017279,0.038992,0.133210,0.125685,-0.061881,-0.007971,-0.010635,-0.019254,0.025751,-0.018852,-0.071324,-0.077275,-0.180433,-0.064782,-0.115584,0.054426,8.053216e-03,0.029098,0.275838,0.084256,0.205182,0.012719,1.654998e-10,0.384855


In [13]:
sample_data.shape


(419, 28)

In [14]:
sample_data.isnull().sum()

Sample_ID                   0
ssGSEA_patient_epi_score    0
ssGSEA_patient_mes_score    0
MEblue                      0
MEturquoise                 0
MEyellow                    0
MEgreen                     0
MEmagenta                   0
MEbrown                     0
MEsalmon                    0
MElightcyan                 0
MEblack                     0
MEgreenyellow               0
MEred                       0
MEmidnightblue              0
MEtan                       0
MEcyan                      0
MEpink                      0
MEpurple                    0
MEgrey                      0
Bcells                      0
CAFs                        0
CD4_Tcells                  0
CD8_Tcells                  0
Endothelial                 0
Macrophages                 0
NK_cells                    0
Other_cells                 0
dtype: int64

In [15]:
x_train, x_test, y_train, y_test = train_test_split(sample_data, sample_metadata, 
                                   random_state = 123,  
                                   test_size = 0.3,  
                                   shuffle = True) 


In [17]:
x_train.head()

,Sample_ID,ssGSEA_patient_epi_score,ssGSEA_patient_mes_score,MEblue,MEturquoise,MEyellow,MEgreen,MEmagenta,MEbrown,MEsalmon,MElightcyan,MEblack,MEgreenyellow,MEred,MEmidnightblue,MEtan,MEcyan,MEpink,MEpurple,MEgrey,Bcells,CAFs,CD4_Tcells,CD8_Tcells,Endothelial,Macrophages,NK_cells,Other_cells
11,mel013,0.330344,0.565402,-0.007634,0.033754,-0.018994,-0.026662,-0.015580,-0.015461,-0.005218,-0.004197,0.069747,0.030789,-0.036769,-0.034747,-0.033372,-0.008595,0.013374,0.027366,-0.029941,4.600864e-10,0.019737,0.250741,3.013406e-07,0.129555,0.014518,2.724409e-05,0.585421
24,mel027,0.288616,0.612249,-0.087314,0.001992,0.033883,0.022042,-0.022838,0.015602,-0.000278,0.225600,0.045754,0.031893,0.080397,0.089682,0.044820,-0.059646,0.044534,0.044857,-0.052411,6.748901e-02,0.027110,0.229437,1.209685e-01,0.134396,0.013154,6.618052e-08,0.407446
36,mel040,0.196585,0.490855,0.035476,0.021721,-0.064626,-0.072675,-0.041236,0.001885,-0.066529,0.009002,-0.024642,-0.075608,0.029021,0.033126,0.042922,-0.016094,0.024709,0.029008,-0.018370,6.594533e-07,0.019088,0.106623,2.545346e-02,0.127699,0.007771,2.916608e-09,0.713365
291,mel326,0.167399,0.601622,-0.003984,0.039347,0.018409,-0.028231,-0.032717,-0.008551,-0.001186,-0.017588,-0.031455,-0.019307,-0.043792,-0.109984,-0.067276,0.078259,0.055563,0.052036,-0.081607,6.506857e-02,0.021529,0.242292,8.078320e-02,0.116407,0.012982,1.228716e-09,0.460939
15,mel018,0.221638,0.512973,0.081921,-0.005906,-0.020872,0.001026,-0.023712,-0.012104,-0.043334,0.009306,-0.002607,0.007138,-0.001344,0.069763,0.079297,-0.135761,-0.090518,0.015527,0.027966,2.788258e-08,0.019066,0.157192,5.062320e-02,0.141009,0.012138,3.703675e-09,0.619971


In [19]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_shape=(27,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

2024-02-09 11:50:04.423802: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-02-09 11:50:04.423822: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 128.00 GB
2024-02-09 11:50:04.423830: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 48.00 GB
2024-02-09 11:50:04.423888: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-09 11:50:04.423926: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [20]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
y_train["tumor_descriptor"].head()

191    Metastatic
385    Metastatic
179    Metastatic
162    Metastatic
160    Metastatic
Name: tumor_descriptor, dtype: object

In [ ]:
# fit the keras model on the dataset
model.fit(x_train.iloc[:,1:28], y_train["tumor_descriptor"], epochs=300, batch_size=20)